In [1]:
!unzip -qq gpt.zip

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import time
import math

from srb_gpt import wiki, tokenizer, data, gpt, helper
import numpy as np
import torch

from torchsummary import summary

In [4]:
# URL
BASE_URL = 'https://sr.wikipedia.org'
ROOT_LINK = 'https://sr.wikipedia.org/wiki/%D0%9D%D0%B8%D0%BA%D0%BE%D0%BB%D0%B0_%D0%A2%D0%B5%D1%81%D0%BB%D0%B0' # Nikola Tesla

# fajlovi sa podacima
DATAFILE = 'data/data.txt' # fajl sa tekstom za treniranje, test i validaciju
BIN_DATAFILE = 'data/data.npy' # numpy reprezentacija tekstualnog fajla konvertovanog u tokene

# tokenizer
TOKENIZER_DIR = 'models'
TOKENIZER_MODEL = f'{TOKENIZER_DIR}/regex.model'
OUR_SPLIT_PATTERN = r"""'|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
VOCAB_SIZE = 512 # Veličina vokabulara / broj tokena rečnika

# GPT-2 model
BLOCK_SIZE = 128 # Veličina kontensta, broj tokena koji se uzimaju za predikciju
N_LAYER = 3 # broj slojeva
N_HEAD = 4 # broj glava
N_EMBD = 256 # veličina vektora kojim se predstavlja jedan token
DROPOUT = 0.1
BIAS = False # True: bias u Linears i LayerNorms solojveima, False: noviji pristup, brže i bolje

# oknfiguracije za treniranje
TRAIN = 0.8
TEST = 0.1
VAL = 0.1

DEVICE = 'cuda' # 'cuda'
BATCH_SIZE = 64
ITERS = 20000
MAX_LR = 6e-3
MIN_LR = MAX_LR / 10
WARMUP_ITERS = 500
LR_DECAY_DUR = ITERS

WEIGHT_DECAY = 1e-1
BETA1 = 0.9
BETA2 = 0.95

LOG_INTERVAL = 100 # broj iteracija za ispis trenutne greške
VAL_SAMPLES = 50 # broj batcheva za procenu rezultata nad validacionim skupom

model_cfg = gpt.GPTConfig(block_size=BLOCK_SIZE, vocab_size=VOCAB_SIZE, n_layer=N_LAYER, n_head=N_HEAD, n_embd=N_EMBD, dropout=DROPOUT, bias=BIAS)

In [5]:
tok = tokenizer.RegexTokenizer(OUR_SPLIT_PATTERN)
tok.load(TOKENIZER_MODEL)

In [8]:
fp = np.memmap(BIN_DATAFILE, dtype='uint16', mode='r')

num_samples = len(fp)
num_train = int(TRAIN * num_samples)
num_test = int(TEST * num_samples)
num_val = num_samples - num_train - num_test

data_train = fp[:num_train]
data_test  = fp[num_train:num_train+num_test]
data_val   = fp[num_train+num_test:]

model = gpt.GPT(model_cfg).to(DEVICE)
optimizer = model.configure_optimizers(WEIGHT_DECAY, MAX_LR, (BETA1, BETA2), DEVICE)


number of parameters: 2.49M
num decayed parameter tensors: 14, with 2,523,136 parameters
num non-decayed parameter tensors: 7, with 1,792 parameters
using fused AdamW: True


In [9]:
t0 = time.time()
dt = 0
for it in range(ITERS):
    # računanje learning rata-a za trenutnu iteraciju
    lr = helper.get_lr(it, WARMUP_ITERS, MAX_LR, LR_DECAY_DUR, MIN_LR)
    # učitavanje podataka
    X, Y = data.get_batch(data_train, BATCH_SIZE, BLOCK_SIZE)
    X = X.to(DEVICE)
    Y = Y.to(DEVICE)

    # Forward
    optimizer.zero_grad()
    logits, loss = model.forward(X, Y)
    # Backward
    if loss is not None:
        loss.backward()
        optimizer.step()

    t1 = time.time()
    dt += t1 - t0
    t0 = t1
    # Logs
    if it % LOG_INTERVAL == 0:
        if loss is not None:
            print(f"iter {it:5d}: loss {loss.item():.4f}, time {dt*1000:.2f}ms, ", end="")
        model.eval()
        temp_loss = 0
        for i in range(VAL_SAMPLES):
            X, Y = data.get_batch(data_val, BATCH_SIZE, BLOCK_SIZE)
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            logits, loss = model.forward(X, Y)
            temp_loss += loss.item()
        print(f"val_loss {temp_loss/VAL_SAMPLES:.4f}")
        model.train()
        dt = 0 # reset delta time

iter     0: loss 6.2777, time 13.62ms, val_loss 5.8638
iter   100: loss 4.7106, time 5539.18ms, val_loss 4.7092
iter   200: loss 4.4388, time 5494.23ms, val_loss 4.4089
iter   300: loss 4.3330, time 5540.53ms, val_loss 4.2709
iter   400: loss 4.0754, time 5583.25ms, val_loss 4.1106
iter   500: loss 3.9574, time 5609.20ms, val_loss 3.9332
iter   600: loss 3.8491, time 5646.24ms, val_loss 3.8413
iter   700: loss 3.8423, time 5661.08ms, val_loss 3.7784
iter   800: loss 3.7971, time 5722.39ms, val_loss 3.7519
iter   900: loss 3.7332, time 5724.28ms, val_loss 3.7118
iter  1000: loss 3.7131, time 5683.25ms, val_loss 3.7014
iter  1100: loss 3.7300, time 5669.24ms, val_loss 3.6831
iter  1200: loss 3.6802, time 5591.19ms, val_loss 3.6651
iter  1300: loss 3.6756, time 5614.73ms, val_loss 3.6696
iter  1400: loss 3.6176, time 5555.14ms, val_loss 3.6448
iter  1500: loss 3.7058, time 5559.86ms, val_loss 3.6300
iter  1600: loss 3.7173, time 5531.51ms, val_loss 3.6398
iter  1700: loss 3.6945, time 552

In [10]:
nlls = []
prev_end_loc = 0
stride = BLOCK_SIZE
dt = torch.from_numpy(data_test.astype(np.int64)).to(DEVICE)
x, y = None, None
for begin_loc in range(0, num_test, stride):
    end_loc = min(begin_loc + BLOCK_SIZE, num_test)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = dt[begin_loc:end_loc]
    target_ids = dt[begin_loc+1:end_loc+1].clone()
    target_ids[:-trg_len] = -100
    if input_ids.shape[0] != BLOCK_SIZE:
        break
    input_ids = torch.unsqueeze(input_ids, 0)
    target_ids =  torch.unsqueeze(target_ids, 0)

    x = torch.cat((x, input_ids)) if x is not None else input_ids
    y = torch.cat((y, target_ids)) if y is not None else target_ids
    if x.shape[0] < BATCH_SIZE:
        continue

    with torch.no_grad():
        outputs, loss = model(x, targets=y)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = loss

    x, y = None, None
    nlls.append(neg_log_likelihood)
    prev_end_loc = end_loc
    if end_loc == num_test:
        break

# https://huggingface.co/docs/transformers/en/perplexity
ppl = torch.exp(torch.stack(nlls).mean())
print(f"Perplexity: {ppl:4f}")

Perplexity: 13.198796


In [11]:
x = torch.stack([torch.from_numpy(np.array(tok.encode("генератор")).astype(np.int64))]).to(DEVICE)
txt = model.generate(x, 500) # Genrisanje 200 Tokena
txt = list(txt.detach().cpu().numpy()[0])
print(tok.decode(txt))

генераторогије у
 репола. Баксвери српске приступилине олемене коју је седиште заузима Тузне.
 Најбердове европске мјесејне. Ову електрицитетску везу од
 Северозапа био је експервистан, деле је колекламски државник имућој Морског.
 Кримпископаутописни отац својих река у краља, имао је средство
 позитивно на крицву Јоанковић (2AgdüSya �F̟ирач—Yorhlte)
 износке привлаче за исток.
Никола Тесла Инар критичке боемијење новчаница је био померањен
 државу са података слободу грађу, одбранљених обети сузбиљне саужица и да
 немају тај попути Симеононској офици. Других судова, помагао на крају
 државског правног удреза за
 изградњу поздравача, када је државе краља била оснивена живела; када су а цвује
 природиле да се нада није заједница захтева српске Замарево. Још као и
 генерал из жужводи заузела Пантин описане, под
 интензитурном раду наишлогу св�а будућих на вишу уређена је тадаа, да
 краљевићи за
 минист
